In [1]:
%load_ext autoreload
%autoreload 2

from source.read_HAR_dataset import read_har_dataset, har_dimensions, har_activities, har_activities_map, har_ind_IDS
from source.utils import  filter_dimensions
from source.tserie import TSerie
from source.utils import classify_dataset
from itertools import chain, combinations
from sklearn.ensemble import AdaBoostClassifier
from source.utils import idsStd
from sklearn import metrics
import numpy as np
from sklearn import svm
import umap

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

Z_SCORE_NORM = True
DATASET = 'HAR-UML20'
KFOLDS = 8
N_TESTS = 2
METRIC  = 'braycurtis'
N_COMPONENTS_LIST = [16, 32, 64]
N_DIMS_NAMES = ['Acc', 'Gyro', 'Mag']
N_DIMENSIONS = [
    [
        'Accelerometer-X',	
        'Accelerometer-Y',	
        'Accelerometer-Z',
    ],
    [
        'Gyrometer-X',
        'Gyrometer-Y',
        'Gyrometer-Z',
    ],
    [
        'Magnetometer-X',
        'Magnetometer-Y',
        'Magnetometer-Z'
    ]
]

/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
results_map = {}
for N_COMPONENTS in N_COMPONENTS_LIST:
    components_map = {}

    for k in range(KFOLDS):
        if DATASET == 'HAR-UML20':
            all_ids = har_ind_IDS
            test_ids = all_ids[k: k + N_TESTS]
            train_ids = all_ids[:k] + all_ids[k + N_TESTS:]        
            
            data = read_har_dataset('./datasets/HAR-UML20/', train_ids=train_ids, test_ids=test_ids, val_ids=[])
            ids_train, X_train, y_train, I_train, train_kcal_MET = data['train']
            # ids_val, X_val, y_val, I_val, val_kcal_MET = data['val']
            ids_test, X_test, y_test, I_test, test_kcal_MET = data['test']
            
            all_dimensions = har_dimensions
            activities_map = har_activities_map
        
        train_embeddings = []
        test_embeddings = []
        train_ind = train_ids
        test_ind = test_ids
        
        ind_std_train = idsStd(train_ids , X_train, I_train)
        ind_std_test = idsStd(test_ids, X_test, I_test)
        
        if Z_SCORE_NORM:
            N_tr, T, D = X_train.shape
            N_te, T, D = X_test.shape
            X_train_sh = np.zeros(X_train.shape)
            X_test_sh = np.zeros(X_test.shape)
            for i in range(N_tr):
                
                for k in range(D):
                    mag = np.mean(X_train[i, :, k], axis = 0)
                    indice = np.where(train_ind ==  I_train[i])[0][0]
                    std = ind_std_train[indice][k] * 6
                    X_train_sh[i, :, k] = (X_train[i, :, k] - mag) / std
                    # X_train_sh[i, :, k] = np.concatenate([[0], fft(X_train[i, :, k])[1:]])

            for i in range(N_te):
                for k in range(D):
                    mag = np.mean(X_test[i, :, k], axis = 0)
                    indice = np.where(test_ind ==  I_test[i])[0][0]
                    std = ind_std_test[indice][k] * 6
                    X_test_sh[i, :, k] = (X_test[i, :, k] - mag) / std
                    # X_test_sh[i, :, k] = fft(X_test[i, :, k])[0:]
        else:
            X_train_sh = np.copy(X_train)
            X_test_sh = np.copy(X_test)
        
        X_train = X_train_sh
        X_test = X_test_sh
        
        for dimensions in N_DIMENSIONS:    
            X_train_f = filter_dimensions(X_train, all_dimensions, dimensions)
            X_test_f = filter_dimensions(X_test, all_dimensions, dimensions)
            
            

            mts_train = TSerie(X = X_train_f, y = y_train, I = I_train, dimensions = dimensions, classLabels=activities_map)
            mts_test = TSerie(X = X_test_f, y = y_test, I = I_test, dimensions = dimensions, classLabels=activities_map)
            
            mts_train.folding_features_v2()
            mts_test.folding_features_v2()
            
            reducer = umap.UMAP(n_components=N_COMPONENTS, metric=METRIC)
            
            embeddings_train = reducer.fit_transform(mts_train.features, y = mts_train.y)
            embeddings_test = reducer.transform(mts_test.features)
            
            train_embeddings.append(embeddings_train)
            test_embeddings.append(embeddings_test)
        
        names_comb = []
        embeddings_comb = []
        
        for i, combo in enumerate(powerset(list(range(len(N_DIMS_NAMES)))), 1):
            indexes = list(combo)
            name = ''
            train_embedding = []
            test_embedding = []
            if len(indexes) == 0:
                continue
            for ind in indexes:
                name = name + ' ' + N_DIMS_NAMES[ind]
                if len(train_embedding) == 0:
                    train_embedding = train_embeddings[ind]
                    test_embedding = test_embeddings[ind]
                else:
                    train_embedding = np.concatenate([train_embedding, train_embeddings[ind]], axis=1) 
                    test_embedding = np.concatenate([test_embedding, test_embeddings[ind]], axis=1) 
            
            names_comb.append(name)
            embeddings_comb.append((train_embedding, test_embedding))
        
        print(names_comb)
        # print(embeddings_comb)
        
        for j in range(len(names_comb)):
            name = names_comb[j]
            clf = AdaBoostClassifier()
            # clf = svm.SVC()
            train_feat, test_feat = embeddings_comb[j]
            clf.fit(train_feat, mts_train.y)
            pred_train, pred_test = clf.predict(train_feat), clf.predict(test_feat)
            
            f1_tr = metrics.f1_score(mts_train.y, pred_train, average='weighted')
            f1_te = metrics.f1_score(mts_test.y, pred_test, average='weighted')
            
            if name not in components_map:
                components_map[name] = [(f1_tr, f1_te)]
            else:
                components_map[name] = components_map[name] + [(f1_tr, f1_te)]
    
    results_map[N_COMPONENTS] = components_map
            
        

IDS: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Train IDS: [2, 3, 4, 5, 6, 7, 8, 9]
Test IDS: [0, 1]
Val IDS: []
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
[' Acc', ' Gyro', ' Mag', ' Acc Gyro', ' Acc Mag', ' Gyro Mag', ' Acc Gyro Mag']


/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A colu

IDS: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Train IDS: [0, 3, 4, 5, 6, 7, 8, 9]
Test IDS: [1, 2]
Val IDS: []
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
[' Acc', ' Gyro', ' Mag', ' Acc Gyro', ' Acc Mag', ' Gyro Mag', ' Acc Gyro Mag']


...

IDS: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Train IDS: [0, 1, 4, 5, 6, 7, 8, 9]
Test IDS: [2, 3]
Val IDS: []
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
[' Acc', ' Gyro', ' Mag', ' Acc Gyro', ' Acc Mag', ' Gyro Mag', ' Acc Gyro Mag']


/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A colu

IDS: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Train IDS: [0, 1, 2, 5, 6, 7, 8, 9]
Test IDS: [3, 4]
Val IDS: []
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
Loaded mts - N: 6300, T: 200, D: 3 
Loaded mts - N: 2100, T: 200, D: 3 
Features shape: (6300, 600)
Features shape: (2100, 600)
[' Acc', ' Gyro', ' Mag', ' Acc Gyro', ' Acc Mag', ' Gyro Mag', ' Acc Gyro Mag']


/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A colu

IDS: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Train IDS: [0, 1, 2, 3, 6, 7, 8, 9]
Test IDS: [4, 5]
Val IDS: []
Loaded mts - N: 6720, T: 200, D: 3 
Loaded mts - N: 1680, T: 200, D: 3 
Features shape: (6720, 600)
Features shape: (1680, 600)


In [ ]:
results_map

{16: {' Acc': [(0.9955566813166441, 0.6735205396130985),
   (0.9938148755933354, 0.6322372749073741),
   (0.9912735023484646, 0.7099475232507996),
   (0.991595843126515, 0.6676908374200788),
   (0.994346959904575, 0.6850280753318788),
   (0.9939783649013338, 0.7340945605897826),
   (0.9927207150462514, 0.7465406827069326),
   (0.9946826845083593, 0.7320930771342221)],
  ' Gyro': [(0.9952367345690227, 0.618143706140068),
   (0.9950812517336917, 0.6086307872335724),
   (0.9938116161793441, 0.676463682974813),
   (0.9941300110403164, 0.6279234006376277),
   (0.9944959215244671, 0.5767746177277472),
   (0.9938420148361093, 0.6831687581087855),
   (0.9938409547488346, 0.6880501648743427),
   (0.993978856299892, 0.733990361783458)],
  ' Mag': [(0.9901648216217049, 0.5039711839282035),
   (0.9898385957449184, 0.47848757269689207),
   (0.9884223026936821, 0.5091998530171201),
   (0.9871467150591123, 0.39142287191096437),
   (0.989739185906364, 0.3794383967376328),
   (0.9839091087363759, 0.431

In [ ]:
import csv

path = 'har_kfold_adaBoost.csv'
with open(path, 'w', newline='') as csvfile:
    row = ['Sensors']
    row = row + [str(com) for com in N_COMPONENTS_LIST]
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(row)
    for name in names_comb:
        row = [name]
        for comp in N_COMPONENTS_LIST:
            mean_tr = np.array([ f1[0] for f1 in results_map[comp][name]]).mean()
            stds_tr = np.array([ f1[0] for f1 in results_map[comp][name]]).std()
            mean_te = np.array([ f1[1] for f1 in results_map[comp][name]]).mean()
            stds_te = np.array([ f1[1] for f1 in results_map[comp][name]]).std()
            
            cell_str = '{:.3f} ({:.3f}) - {:.3f} ({:.3f})'.format(mean_tr, stds_tr, mean_te, stds_te)
            row.append(cell_str)
            
        spamwriter.writerow(row)
            

In [5]:
names_comb

[' Acc',
 ' Gyro',
 ' Mag',
 ' Acc Gyro',
 ' Acc Mag',
 ' Gyro Mag',
 ' Acc Gyro Mag']